# Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import MessagePassing, BatchNorm
from torch_geometric.utils import to_networkx, subgraph, k_hop_subgraph
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import random

In [ ]:
# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load Cora Dataset

In [ ]:
dataset = Planetoid(root="data/Cora", name="Cora")
data = dataset[0]

In [ ]:
node_of_interest = random.randint(0, data.num_nodes - 1)
print("Node of Interest:", node_of_interest)

# extract 2-hop subgraph
subset_nodes, sub_edge_index, _, _ = k_hop_subgraph(
    node_of_interest, 
    num_hops=2, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)

sub_x = data.x[subset_nodes]
sub_y = data.y[subset_nodes]

print("Subgraph Nodes:", sub_x.size(0))

In [ ]:
G = to_networkx(
    Data(x=sub_x, edge_index=sub_edge_index), 
    to_undirected=True
)

plt.figure(figsize=(6,6))
nx.draw(G, node_size=50)
plt.title("Subgraph (2-hop Neighborhood)")
plt.show()

In [ ]:
#Jumlah Node
data.num_nodes

In [ ]:
#jumlah edge
data.num_edges

In [ ]:
#dimensi fitur
data.num_features

In [ ]:
#jumlah keasl
dataset.num_classes

In [ ]:
#nama kelas
data.y.unique()

In [ ]:
#nodes belong to classes
data.y.bincount()

In [ ]:
class_counts = data.y.bincount()
for class_idx in range(len(class_counts)):
    print(f"Kelas {class_idx}: {class_counts[class_idx].item()} nodes")

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.bar(range(len(class_counts)), class_counts.numpy(), color='skyblue')
plt.title('Distribusi Node per Kelas')
plt.xlabel('Kelas')
plt.ylabel('Jumlah Node')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.pie(class_counts.numpy(), labels=[f'Kelas {i}' for i in range(len(class_counts))], 
        autopct='%1.1f%%', startangle=90)
plt.title('Proporsi Kelas')

plt.tight_layout()
plt.show()

In [ ]:
#train nodes
data.train_mask.sum().item()

In [ ]:
print(f"\nTrain ratio: {data.train_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#validation nodes
data.val_mask.sum().item()

In [ ]:
print(f"Validation ratio: {data.val_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#test nodes
data.test_mask.sum().item()

In [ ]:
print(f"Test ratio: {data.test_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
data.edge_index.shape

In [ ]:
data.edge_index[:, :5]

In [ ]:
data.x[0, :10]

# PRA-PROSES (Message Passing Layer)

$h_u^{(k)} = \sum W_{\text{msg}} \cdot h_u^{(k-1)} + W_{\text{self}} \cdot h_v^{(k-1)}$

 di mana,
 1. **k** adalah urutan lapisan GNN.
 2. **Wmsg** adalah bobot yang diakses bersama dengan neighborhood nodes.
 3. **Wself** adalah bobot milik node asal (v).

In [ ]:
class CustomMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.W_msg = nn.Linear(in_channels, out_channels)  # For neighbors
        self.W_self = nn.Linear(in_channels, out_channels) # For self
        
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j):
        return self.W_msg(x_j)  # W_msg * h_u
    
    def update(self, aggr_out, x):
        return aggr_out + self.W_self(x)  # Σ + W_self * h_v

In [ ]:
# TEST MESSAGE PASSING

test_mp = CustomMessagePassing(in_channels=16, out_channels=32)
test_mp

In [ ]:
test_mp.W_msg.weight.shape

In [ ]:
test_mp.W_self.weight.shape

# Intra-layer GNN Block

$$
\mathbf{h}_v^{(k)} =
\text{ACT}\!\left(
    \text{Dropout}\!\left(
        \text{BatchNorm}\!\left(
            \mathbf{W}^{(k-1)}\,\mathbf{h}_v^{(k-1)} + \mathbf{b}^{(k-1)}
        \right)
    \right)
\right)
+ \mathbf{h}_v^{(k-1)}
$$


di mana:

1. **ACT** adalah fungsi aktivasi yang Anda pilih.
2. **BatchNorm** adalah `1D-Batch Normalization`.


## DEFINE SINGLE GNN LAYER 

In [ ]:
class GNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.5):
        super().__init__()
        
        #Add message passing layer!
        self.message_passing = CustomMessagePassing(in_channels, out_channels)
        
        # Linear transformation
        self.linear = nn.Linear(out_channels, out_channels)  # Changed from in_channels!
        
        # Required components
        self.batch_norm = BatchNorm(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()
        
        #use residual
        if in_channels != out_channels:
            self.residual = nn.Linear(in_channels, out_channels)
        else:
            self.residual = nn.Identity()
    
    def forward(self, x, edge_index):
        x_input = x
        
        #USE MESSAGE PASSING!
        x_mp = self.message_passing(x, edge_index)  # This was missing!
        
        # Process
        x_linear = self.linear(x_mp)  # Changed from x_mp to x
        x_bn = self.batch_norm(x_linear)
        x_drop = self.dropout(x_bn)
        x_act = self.activation(x_drop)
        
        # Residual
        x_residual = self.residual(x_input)
        return x_act + x_residual  # This is correct now

## Test GNN Layer

In [ ]:
test_layer = GNNLayer(in_channels=16, out_channels=32)
test_layer

In [ ]:
test_layer.batch_norm

In [ ]:
test_layer.dropout.p

In [ ]:
test_layer.activation

# Full Model with k>1 Layers

## Build Model

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        
        
        self.input_proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        
        # addmore layers with message passing
        self.layer1 = GNNLayer(hidden_channels, hidden_channels, dropout=0.5)
        self.layer2 = GNNLayer(hidden_channels, hidden_channels, dropout=0.5)
        self.layer3 = GNNLayer(hidden_channels, hidden_channels, dropout=0.3)
        self.layer4 = GNNLayer(hidden_channels, out_channels, dropout=0.3)  # Keep residual!
        
        # Classifier
        self.classifier = nn.Linear(out_channels, out_channels)
    
    def forward(self, x, edge_index):
        # Project features
        x = self.input_proj(x)
        
        # Multiple GNN layers
        x = self.layer1(x, edge_index)
        x = F.relu(x)
        x = self.layer2(x, edge_index)
        x = F.relu(x)
        x = self.layer3(x, edge_index)
        x = F.relu(x)
        x = self.layer4(x, edge_index)  # Now with message passing
        
        # Classify
        x = self.classifier(x)
        return F.log_softmax(x, dim=-1)

## Initialize Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# model = GNNModel(
#     in_channels=data.num_features,
#     hidden_channels=512,
#     out_channels=dataset.num_classes
# ).to(device)

# print(f"Model created on {device}")

In [ ]:
model = GNNModel(
    in_channels=data.num_features,
    hidden_channels=256,  # Better for Cora
    out_channels=dataset.num_classes
).to(device)

## CHECK MODEL PARAMETERS

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(f"Total parameters: {total_params:,}")

In [ ]:
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
print("\nParameter shapes:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name:30} {tuple(param.shape)}")

## SET UP OPTIMIZER

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=10
# )

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

In [ ]:
# More patience
patience = 50  # Was 20
epochs = 300   # Was 150

In [ ]:
print(f"Learning rate: {optimizer.param_groups[0]['lr']}")

In [ ]:
print(f"Weight decay: {optimizer.param_groups[0]['weight_decay']}")

In [ ]:
# move data to device
data = data.to(device)
device
data.x.device
data.edge_index.device

# Show Model Structure

In [ ]:
model

In [ ]:
for name, module in model.named_children():
    print(f"\n{name}: {module}")

# Training

In [ ]:
print(f"Optimizer: {optimizer}")
print(f"Scheduler: {scheduler}")
print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
print(f"Epochs: 150")
print(f"Early stopping patience: 20")
print(f"Device: {device}")

## RAINING LOOP WITH VALIDATION LOSS

In [ ]:
# def get_validation_loss(model, data):
#     model.eval()
#     with torch.no_grad():
#         out = model(data.x, data.edge_index)
#         loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
#     return loss.item()

# # Then track both losses:
# val_losses = []
# for epoch in range(epochs):
#     train_loss = train_one_epoch(...)
#     val_loss = get_validation_loss(model, data)
#     val_losses.append(val_loss)  # We're missing this!

## SEtup

In [ ]:
train_losses = []
val_accuracies = []
test_accuracies = []

### DEFINE TRAINING STEP FUNCTION

In [ ]:
def train_one_epoch(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
    
    optimizer.step()
    return loss.item()

In [ ]:
def evaluate_full(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        
        # Losses
        train_loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask]).item()
        val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask]).item()
        
        # Accuracies
        train_acc = (out[data.train_mask].argmax(dim=1) == data.y[data.train_mask]).float().mean().item()
        val_acc = (out[data.val_mask].argmax(dim=1) == data.y[data.val_mask]).float().mean().item()
        test_acc = (out[data.test_mask].argmax(dim=1) == data.y[data.test_mask]).float().mean().item()
    
    return train_loss, val_loss, train_acc, val_acc, test_acc

### DEFINE EVALUATION FUNCTION

In [ ]:
def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[mask].argmax(dim=1)
        correct = (pred == data.y[mask]).sum().item()
        total = mask.sum().item()
        accuracy = correct / total
    return accuracy

## RUN EPOCH

In [ ]:
best_val_acc = 0
patience_counter = 0
patience = 20


for epoch in range(150):
    # Train
    loss = train_one_epoch(model, data, optimizer)
    train_losses.append(loss)
    
    # Evaluate
    val_acc = evaluate(model, data, data.val_mask)
    test_acc = evaluate(model, data, data.test_mask)
    val_accuracies.append(val_acc)
    test_accuracies.append(test_acc)
    
    # Update scheduler
    scheduler.step(loss)
    
    # Check best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
    
    # Print progress
    if epoch % 10 == 0:
        print(f'Epoch {epoch:3d}: Loss={loss:.4f}, Val={val_acc:.4f}, Test={test_acc:.4f}')
    
    # Early stopping
    if patience_counter >= patience:
        print(f'Early stopping at epoch {epoch}')
        break

### Load best model

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

## PLOT

### Training loss

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, linewidth=2)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
# Plot last 50 losses to see detail
plt.plot(train_losses[-50:], linewidth=2, color='red')
plt.title('Training Loss (Last 50 epochs)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### ACCURACIES

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(val_accuracies, label='Validation', linewidth=2)
plt.plot(test_accuracies, label='Test', linewidth=2)
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
# Moving average
window = 5
val_ma = np.convolve(val_accuracies, np.ones(window)/window, mode='valid')
plt.plot(val_ma, linewidth=2, color='green')
plt.title(f'Validation Accuracy (MA{window})')
plt.xlabel('Epoch')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Prediction

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(data.x, data.edge_index)
    pred_classes = predictions.argmax(dim=1)
    pred_probs = torch.exp(predictions)

In [ ]:
predictions.shape

In [ ]:
pred_classes[0].item()

In [ ]:
pred_probs[0].cpu().numpy().round(3)

In [ ]:
cm = confusion_matrix(
    data.y[data.test_mask].cpu(),
    pred_classes[data.test_mask].cpu()
)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (Test Set)')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
test_mask = data.test_mask
y_true = data.y[test_mask].cpu()
y_pred = pred_classes[test_mask].cpu()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, 
                          target_names=[f'Class {i}' for i in range(7)]))

In [ ]:
test_indices = torch.where(data.test_mask)[0][:10]

In [ ]:
for i, idx in enumerate(test_indices):
    true_class = data.y[idx].item()
    pred_class = pred_classes[idx].item()
    confidence = pred_probs[idx].max().item()
    
    print(f"Node {idx:4d}: True={true_class}, Pred={pred_class}, "
          f"Conf={confidence:.3f} {'correct' if true_class == pred_class else 'incorrect'}")
    
    # Show top 3 probabilities for first 3 examples
    if i < 3:
        probs = pred_probs[idx].cpu().numpy()
        top3_idx = np.argsort(probs)[-3:][::-1]
        for j, class_idx in enumerate(top3_idx):
            print(f"    Top-{j+1}: Class {class_idx} = {probs[class_idx]:.3f}")

In [ ]:
model.eval()
with torch.no_grad():
    embeddings = model(data.x, data.edge_index).cpu().numpy()

In [ ]:
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                     c=data.y.cpu().numpy(), cmap='tab10', 
                     alpha=0.7, s=20)
plt.colorbar(scatter, label='Class')
plt.title('Node Embeddings (t-SNE)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Pick some training nodes
train_nodes = torch.where(data.train_mask)[0][:5].cpu()
print(f"Selected nodes of interest: {train_nodes.tolist()}")

# Create 2-hop subgraph
subset, edge_index, _, _ = k_hop_subgraph(
    node_idx=train_nodes,
    num_hops=2,
    edge_index=data.edge_index.cpu(),
    relabel_nodes=True
)

print(f"Subgraph size: {len(subset)} nodes (original: {len(train_nodes)})")
print(f"Subgraph edges: {edge_index.shape[1]}")
print(f"Original indices of sampled nodes: {subset[:10].tolist()}...")

In [ ]:
final_val_acc = evaluate(model, data, data.val_mask)
final_test_acc = evaluate(model, data, data.test_mask)

print(f"Validation Accuracy: {final_val_acc:.4f}")
print(f"Test Accuracy: {final_test_acc:.4f}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")